In [2]:
import elasticsearch
import requests
from bs4 import BeautifulSoup
import re
from tqdm import tqdm
import pickle
import time
from urllib.parse import urljoin

In [ ]:
es_host = "http://localhost:9200"
es_user = "elastic"
es_password = "1234"

%system docker compose up "-d"

time.sleep(10.) # give docker time to set up

es = elasticsearch.Elasticsearch(
    hosts=es_host,  # "http://localhost:9200"
    basic_auth=(es_user, es_password)   # credentials for basic authentication
)

es.info()

In [4]:
es_host = "http://localhost:9200"
es_user = "elastic"
es_password = "1234"
es = elasticsearch.Elasticsearch(
    hosts=es_host,  # "http://localhost:9200"
    basic_auth=(es_user, es_password)   # credentials for basic authentication
)
es.info()

ObjectApiResponse({'name': 'es01', 'cluster_name': 'es-docker-single-cluster', 'cluster_uuid': 'T9rYYv05RRWXv_qoiKOQCg', 'version': {'number': '8.5.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': 'a846182fa16b4ebfcc89aa3c11a11fd5adf3de04', 'build_date': '2022-11-17T18:56:17.538630285Z', 'build_snapshot': False, 'lucene_version': '9.4.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

In [9]:
def get_everything_via_author(author_link, host, auth,
                              index1, index2, all_ids = set()):
    '''
    get an authors link + name + papers (id)
    get papers id, title, abstract, date + author (via link)
    :param author_link:
    :return:
    '''

    soup = BeautifulSoup(requests.get(author_link).content, "html.parser")
    author_name = soup.h2.text
    author_link = author_link.split("/")[-3]+"/" + author_link.split("/")[-2]
    paper_ids = [id["href"].split(".bib")[0].split("/")[1] for id in soup.find_all("a", attrs={"title":"Export to BibTeX"})]
    requests.post(   # make POST request
    # set target url -> http://localhost:9200/products/_doc
        urljoin(host, f"{index1}/_doc"),
        auth=(auth[0], auth[1]),
        json={
            "name": author_name,
            "url": author_link,
            "papers": paper_ids,
        },
    )
    paper_ids = set(paper_ids).difference(all_ids)
    all_ids = all_ids.union(paper_ids)
    for p in soup.find_all(lambda tag: tag.name == 'span' and
                                   tag.get('class') == ['d-block']):
        p_id = p.strong.a["href"].split("/")[1]
        if not p_id in paper_ids:
            continue
        p_title = p.strong.a.text
        authors_links = [t["href"].split("/")[-3]+"/"+t["href"].split("/")[-2] for t in p.find_all("a", attrs={"href":re.compile(r"/people.*")})]
        abstract_id = "abstract-"+ "--".join(p_id.split("."))
        try:
            p_abstract = soup.find("div", attrs={"id":f"{abstract_id}"}).div.text
        except:
            p_abstract = None
        year = re.findall(r'\d+', p_id[:4])[0]
        if len(year) == 2:
            year = int(year)
            if year < 45:
                year += 2000
            else:
                year += 1900
        else:
            year = int(year)

        requests.post(   # make POST request
    # set target url -> http://localhost:9200/products/_doc
        urljoin(host, f"{index2}/_doc"),
        auth=(auth[0], auth[1]),
        json={
            "id": p_id,
            "title": p_title,
            "date": year,
            "abstract":p_abstract,
            "authors":authors_links
        },
    )
    return all_ids


def get_index_author(authors_links, host, auth, indices, es=es, report = True):
    all_p_ids = set()
    pbar = tqdm(total = len(authors_links)+1)
    status = 0
    while authors_links:
        if (status == 100) & report:
            print(es.cat.indices(format="JSON", s="index"))
            status = 0
        link = authors_links.pop()
        all_p_ids= get_everything_via_author(link, host, auth, index1=indices[0],
                                             index2=indices[1], all_ids=all_p_ids)
        pickle.dump(authors_links, open("remaining_author_links.pkl", 'wb'))
        status += 1
        pbar.update(1)
    pbar.close()
    return None

In [10]:
remaining_persons = pickle.load(open("./remaining_author_links.pkl", 'rb'))
get_index_author(remaining_persons, host=es_host, auth=[es_user, es_password], indices=["author", "paper"])

  0%|          | 100/68222 [00:50<11:11:29,  1.69it/s]

[{'health': 'yellow', 'status': 'open', 'index': 'author', 'uuid': 'UKAkK0IwTvyeSTfwRUFJdw', 'pri': '1', 'rep': '1', 'docs.count': '903', 'docs.deleted': '0', 'store.size': '215.9kb', 'pri.store.size': '215.9kb'}, {'health': 'yellow', 'status': 'open', 'index': 'paper', 'uuid': 'cZCTO_0nT4maewg2cPsYLQ', 'pri': '1', 'rep': '1', 'docs.count': '3034', 'docs.deleted': '0', 'store.size': '3.8mb', 'pri.store.size': '3.8mb'}]


  0%|          | 200/68222 [01:42<8:36:51,  2.19it/s] 

[{'health': 'yellow', 'status': 'open', 'index': 'author', 'uuid': 'UKAkK0IwTvyeSTfwRUFJdw', 'pri': '1', 'rep': '1', 'docs.count': '903', 'docs.deleted': '0', 'store.size': '215.9kb', 'pri.store.size': '215.9kb'}, {'health': 'yellow', 'status': 'open', 'index': 'paper', 'uuid': 'cZCTO_0nT4maewg2cPsYLQ', 'pri': '1', 'rep': '1', 'docs.count': '3034', 'docs.deleted': '0', 'store.size': '4mb', 'pri.store.size': '4mb'}]


  0%|          | 300/68222 [02:23<8:12:41,  2.30it/s]

...

[{'health': 'yellow', 'status': 'open', 'index': 'author', 'uuid': 'UKAkK0IwTvyeSTfwRUFJdw', 'pri': '1', 'rep': '1', 'docs.count': '903', 'docs.deleted': '0', 'store.size': '215.9kb', 'pri.store.size': '215.9kb'}, {'health': 'yellow', 'status': 'open', 'index': 'paper', 'uuid': 'cZCTO_0nT4maewg2cPsYLQ', 'pri': '1', 'rep': '1', 'docs.count': '3034', 'docs.deleted': '0', 'store.size': '4.4mb', 'pri.store.size': '4.4mb'}]


  1%|          | 500/68222 [03:59<9:14:34,  2.04it/s] 

[{'health': 'yellow', 'status': 'open', 'index': 'author', 'uuid': 'UKAkK0IwTvyeSTfwRUFJdw', 'pri': '1', 'rep': '1', 'docs.count': '903', 'docs.deleted': '0', 'store.size': '263.3kb', 'pri.store.size': '263.3kb'}, {'health': 'yellow', 'status': 'open', 'index': 'paper', 'uuid': 'cZCTO_0nT4maewg2cPsYLQ', 'pri': '1', 'rep': '1', 'docs.count': '3034', 'docs.deleted': '0', 'store.size': '4.5mb', 'pri.store.size': '4.5mb'}]


  1%|          | 519/68222 [04:07<7:41:09,  2.45it/s]

KeyboardInterrupt: 

ObjectApiResponse({'took': 545, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 7612, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'paper', '_id': 'arsHZocBjUF5wihvnO4q', '_score': 1.0, '_source': {'id': 'L08-1032', 'title': 'An Exchange Format for Multimodal Annotations', 'date': 2008, 'abstract': 'This paper presents the results of a joint effort of a group of multimodality researchers and tool developers to improve the interoperability between several tools used for the annotation of multimodality. We propose a multimodal annotation exchange format, based on the annotation graph formalism, which is supported by import and export routines in the respective tools.', 'authors': ['t/thomas-schmidt', 's/susan-duncan', 'o/oliver-ehmer', 'j/jeffrey-hoyt', 'm/michael-kipp', 'd/dan-loehr', 'm/magnus-magnusson', 't/travis-rose', 'h/han-sloetjes']}}, {'_index': 'paper', '_id': 'bLsHZocBjUF5wihv9O5T', '_score': 1.0

In [12]:
print(es.cat.indices(format="JSON", s="index"))

[{'health': 'yellow', 'status': 'open', 'index': 'author', 'uuid': 'UKAkK0IwTvyeSTfwRUFJdw', 'pri': '1', 'rep': '1', 'docs.count': '903', 'docs.deleted': '0', 'store.size': '263.3kb', 'pri.store.size': '263.3kb'}, {'health': 'yellow', 'status': 'open', 'index': 'paper', 'uuid': 'cZCTO_0nT4maewg2cPsYLQ', 'pri': '1', 'rep': '1', 'docs.count': '7612', 'docs.deleted': '0', 'store.size': '6mb', 'pri.store.size': '6mb'}]
